In [ ]:
!pip install -q datasets evaluate triton optuna seqeval pylcs

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 3.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 7.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 386.6/386.6 kB 18.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 231.9/231.9 kB 19.7 MB/s eta 0:00:00


In [ ]:
import pandas as pd
import numpy as np
from tqdm import tqdm
import pickle
import os
import pylcs
from typing import Union, List, Optional
from pprint import pprint

import transformers
from transformers import (AutoTokenizer, AutoModelForTokenClassification,
                         pipeline, TrainingArguments, Trainer,
                         DataCollatorForTokenClassification, EarlyStoppingCallback)

import datasets
from datasets import Dataset, Features, Sequence, Value, ClassLabel, load_dataset, load_from_disk, concatenate_datasets

import optuna
import torch
import evaluate
import re
from packaging import version


print(transformers.__version__)

import os
os.environ['WANDB_DISABLED'] = 'true'

task = "ner" # Should be one of "ner", "pos" or "chunk"
# model_checkpoint = "distilbert-base-uncased"
batch_size = 16
seqeval = evaluate.load("seqeval")

4.51.3


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


In [ ]:
pd.set_option("max_colwidth", 100)
pd.set_option("display.max_columns", 100)
pd.set_option("display.max_rows", 100)

In [ ]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


## Making dataset

Тренировочный датасет из датасета [Stucco](https://github.com/stucco/auto-labeled-corpus) создаю локально. Здесь форматирую датасет для тренировки моделью.

In [ ]:
df = pd.read_csv('/content/test_df_cve_dataset_bio.tsv', sep='\t')

In [ ]:
df.head()

,Unnamed: 0,cve_id,words,custom_bio
0,0,CVE-2012-1790,Absolute,O
1,1,0,path,O
2,2,0,traversal,O
3,3,0,vulnerability,O
4,4,0,in,O


In [ ]:
df['words'] = df['words'].fillna(' ')

In [ ]:
ids = []
words = []
tags = []

word_column_name = 'words'
bio_column_name = 'custom_bio'

sentence_id = 0
sentence_word = []
sentence_tag = []
for i, row in tqdm(df.iterrows()):

    if row['cve_id'] != '0':
        if sentence_id == 0:
            sentence_word.append(row[word_column_name])
            sentence_tag.append(row[bio_column_name])
            sentence_id += 1
        else:
            ids.append(sentence_id)
            words.append(sentence_word)
            tags.append(sentence_tag)
            sentence_id += 1
            sentence_word = []
            sentence_tag = []
            sentence_word.append(row[word_column_name])
            sentence_tag.append(row[bio_column_name])
    else:
        sentence_word.append(row[word_column_name])
        sentence_tag.append(row[bio_column_name])

70143it [00:05, 13736.63it/s]


In [ ]:
lbls_in_dataset = [
    'O',
    'B-product',
    'I-product',
    'B-vendor',
    'I-vendor',
    'B-version',
    'I-version',
]

In [ ]:
features=Features(
    {
        "id": Value(dtype='int32', id=None),
        "ner_tags": Sequence(feature=ClassLabel(num_classes=len(lbls_in_dataset), names=list(lbls_in_dataset)), id=None),
        "tokens": Sequence(feature=Value(dtype='string', id=None))
    }
)

ds = Dataset.from_dict({"id": list(range(len(words))),
                        "tokens": words,
                        "ner_tags": tags},
                       features=features)

In [ ]:
ds

Dataset({
    features: ['id', 'tokens', 'ner_tags'],
    num_rows: 1532
})

In [ ]:
ds = ds.train_test_split(
    test_size=0.1, shuffle=True, seed=42,
    # stratify_by_column='ner_tags'
)

AttributeError: 'DatasetDict' object has no attribute 'train_test_split'

In [ ]:
ds.save_to_disk('/content/drive/MyDrive/vkr_pandan/dataset/230425_test')

Saving the dataset (0/1 shards):   0%|          | 0/1532 [00:00<?, ? examples/s]

## Loading tokenizer

In [ ]:
bert_tok = AutoTokenizer.from_pretrained('/content/drive/MyDrive/vkr_pandan/bert_base_220425/best_model', use_fast=True, add_prefix_space=True)

In [ ]:
s = '''
The Contact Form 7 plugin for WordPress is vulnerable to Reflected Cross-Site Scripting via the ‘active-tab’ parameter in all versions up to, and including, 5.9 due to insufficient input sanitization and output escaping. This makes it possible for unauthenticated attackers to inject arbitrary web scripts in pages that execute if they can successfully trick a user into performing an action such as clicking on a link.
'''
r = bert_tok(s)
[bert_tok.decode(x) for x in r.input_ids][:5]

['[CLS]', 'the', 'contact', 'form', '7']

In [ ]:
from transforme        learning_rate=trial.suggest_loguniform('learning_rate', low=1e-5, high=1e-2),
        weight_decay=trial.suggest_loguniform('weight_decay', 1e-3, 0.1),
        num_train_epochs=trial.suggest_int('num_train_epochs', low = 2, high = 5),rs import AutoModelForTokenClassification, TrainingArguments, Trainer, AutoModel

tokenizer = AutoTokenizer.from_pretrained('numind/NuNER-v2.0', use_fast=True, add_prefix_space=True)
# model = AutoModelForTokenClassification.from_pretrained('numind/NuNER-v2.0', num_labels=len(lbls_in_dataset), id2label=id2label, label2id=label2id)
# model = AutoModel.from_pretrained('numind/NuNER-v2.0')

tokenizer_config.json:   0%|          | 0.00/1.21k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/798k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.11M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/957 [00:00<?, ?B/s]

In [ ]:
s = '''
The Contact Form 7 plugin for WordPress is vulnerable to Reflected Cross-Site Scripting via the ‘active-tab’ parameter in all versions up to, and including, 5.9 due to insufficient input sanitization and output escaping. This makes it possible for unauthenticated attackers to inject arbitrary web scripts in pages that execute if they can successfully trick a user into performing an action such as clicking on a link.
'''
r = tokenizer(s)
[tokenizer.decode(x) for x in r.input_ids][:5]

['<s>', ' ', '\n', 'The', ' Contact']

## Preparing dataset


In [ ]:
def tokenize_and_align_labels(examples, tokenizer):
    tokenized_inputs = tokenizer(examples['tokens'], truncation=True, is_split_into_words=True)

    labels = []
    for i, label in enumerate(examples['ner_tags']):
        word_ids = tokenized_inputs.word_ids(batch_index=i)
        previous_index = None
        label_ids = []
        for word_idx in word_ids:
            if word_idx is None:
                label_ids.append(-100)
            else:
                label_ids.append(label[word_idx])
        labels.append(label_ids)

    tokenized_inputs['labels'] = labels
    return tokenized_inputs



In [ ]:
tokenized_ds_test = ds.map(tokenize_and_align_labels, batched=True)

NameError: name 'ds' is not defined

In [ ]:
tokenized_ds_test.save_to_disk('/content/drive/MyDrive/vkr_pandan/dataset/230425_test')

Saving the dataset (0/1 shards):   0%|          | 0/1532 [00:00<?, ? examples/s]

Датасет `230425` создан после добавления малопредставленных версий в обучающий датасет Stucco. Также создана отдельная валидационная выборка `230425_test` для сравнения разных моделей, которая не была загружена в Colab

## Training

In [ ]:
# этот способ перестал работать: видимо, что-то с версиями
# ds_train = load_dataset("/content/drive/MyDrive/vkr_pandan/dataset/230425")
ds_test = load_from_disk('/content/drive/MyDrive/vkr_pandan/dataset/230425_test')

In [ ]:
# создаем тренировочный датасет
ds_tr = load_from_disk('/content/drive/MyDrive/vkr_pandan/dataset/230425/train')
ds_te = load_from_disk('/content/drive/MyDrive/vkr_pandan/dataset/230425/test')
ds_train = datasets.DatasetDict({"train":ds_tr,"test":ds_te})

In [ ]:
lbls_in_dataset = [
    'O',
    'B-product',
    'I-product',
    'B-vendor',
    'I-vendor',
    'B-version',
    'I-version',
]
label2id = {v:i for i, v in enumerate(lbls_in_dataset)}
id2label = {i:v for i, v in enumerate(lbls_in_dataset)}


In [ ]:
tokenized_ds_train = ds_train.map(tokenize_and_align_labels, batched=True)
tokenized_ds_test = ds_test.map(tokenize_and_align_labels, batched=True)

NameError: name 'tokenize_and_align_labels' is not defined

In [ ]:
all_wrds = {}
for ds in [ds_train['train'], ds_train['test'], ds_test]:
    len_sen = {'product': 0, 'vendor': 0, 'version': 0, 'other': 0}
    for sen in ds['ner_tags']:
        for tag in sen:
            if tag == 1 or tag == 2:
                len_sen['product'] += 1
            if tag == 3 or tag == 4:
                len_sen['vendor'] += 1
            if tag == 5 or tag == 6:
                len_sen['version'] += 1
            else:
                len_sen['other'] += 1
    all_wrds[ds] = len_sen

In [ ]:
al = [tokenizer.decode(x) for x in tokenized_ds_test[0]['input_ids']]
[(x,y) for x,y in zip(al, tokenized_ds_test[0]['labels'])][:5]

[('<s>', -100), (' Absolute', 0), (' path', 0), (' travers', 0), ('al', 0)]

In [ ]:
# small_dataset_train = tokenized_ds['train'].select(range(20))
# small_dataset_test = tokenized_ds['test'].select(range(10))


In [ ]:
tokenizer = AutoTokenizer.from_pretrained('numind/NuNER-v2.0', use_fast=True, add_prefix_space=True)
model = AutoModelForTokenClassification.from_pretrained('numind/NuNER-v2.0', num_labels=len(lbls_in_dataset), id2label=id2label, label2id=label2id)

config.json:   0%|          | 0.00/671 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/499M [00:00<?, ?B/s]

Some weights of RobertaForTokenClassification were not initialized from the model checkpoint at numind/NuNER-v2.0 and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
device = torch.device('cuda:0') if torch.cuda.is_available() else torch.device('cpu')
print(f"Using device: {device}")

print("Compiling model for faster training...")
torch.set_float32_matmul_precision('high')
model.to(device)

In [ ]:
# функция метрики
def compute_metrics_custom(p):
    labels_list = list(id2label.values())
    predictions, labels = p
    if not os.path.exists('/content/test/p_trainer.pkl'):
        os.makedirs('/content/test', exist_ok=True)
        with open('/content/test/p_trainer.pkl', 'wb') as f:
            pickle.dump(p, f)
    predictions = np.argmax(predictions, axis=2)


    true_predictions = [
        [labels_list[p] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]
    true_labels = [
        [labels_list[l] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]

    results = seqeval.compute(predictions=true_predictions, references=true_labels)
    return {
        "precision": results["overall_precision"],
        "recall": results["overall_recall"],
        "f1": results["overall_f1"],
        "accuracy": results["overall_accuracy"],
    }

In [ ]:
data_collator = DataCollatorForTokenClassification(tokenizer)

We use RoBERTa-base (Liu et al., 2019) for both the text
encoder and the concept encoder. Our model is trained for
10 epochs on the full 1M sentence dataset where 90% of
sentences are used for training and 10% for validation. We
choose a learning rate lr = 0.00003, batch size= 48, and
temperature before the sigmoid τ = 5. We use AdamW
optimizer (Kingma & Ba, 2015) with β1 = 0.9, β2 =
0.999, ϵ = 10−6, weight decay=0.01, and a linear scheduler
with a warm-up for the first 10% of the training steps.

# Fine-tune with Optuna

In [ ]:
import optuna.logging
optuna.logging.set_verbosity(optuna.logging.INFO)

best_f1 = 0

GDRIVE_DIR = '/content/drive/MyDrive/vkr_pandan/nuner_110525/best_model'

def printer(s):
    print('*'*150, end='\n\n')
    print(s, end='\n\n')
    print('*'*150, end='\n\n')

In [ ]:
!rm /content/drive/MyDrive/vkr_pandan/nuner_110525/best_model_tmp/*

rm: cannot remove '/content/drive/MyDrive/vkr_pandan/nuner_110525/best_model_tmp/*': No such file or directory


In [ ]:
!rm -rf ./best_model

In [ ]:
!rm -rf ./checkpoint*

In [ ]:
def objective(trial: optuna.Trial, model=model):
    global best_f1
    model = model
    model_name = 'nuner_110525_optuna_v1'
    trial_check_dir = f"./checkpoints_trial"

    os.makedirs(trial_check_dir, exist_ok=True)
    args = TrainingArguments(
        # f"{model_name}-finetuned-ner",
        output_dir=trial_check_dir,
        overwrite_output_dir=True,
        eval_strategy = "epoch",
        torch_compile=True,
        # 64\
        per_device_train_batch_size=32,
        per_device_eval_batch_size=32,
        learning_rate=trial.suggest_loguniform('learning_rate', low=1e-5, high=5e-4),
        weight_decay=trial.suggest_loguniform('weight_decay', 1e-3, 0.1),
        num_train_epochs=trial.suggest_int('num_train_epochs', low = 3, high = 8),
        seed=42,
        data_seed=24,
        gradient_accumulation_steps=2,
        warmup_ratio=0.1,
        report_to=None,
        logging_dir="./logs",
        logging_steps=1,
        load_best_model_at_end=True,
        metric_for_best_model="eval_f1",
        # greater_is_better=False,
        save_total_limit=1,
        save_strategy="epoch",  # Changed to match evaluation_strategy
    )

    # early_stopping = EarlyStoppingCallback(
    #     early_stopping_patience=1,  # Stop if F1 decreases for 1 consecutive epoch
    #     early_stopping_threshold=0.001
    # )

    trainer = Trainer(
        model,
        args,
        train_dataset=tokenized_ds['train'],
        eval_dataset=tokenized_ds['test'],
        # train_dataset=small_dataset_train,
        # eval_dataset=small_dataset_test,
        data_collator=data_collator,
        compute_metrics=compute_metrics_custom,
        tokenizer=tokenizer,
        # callbacks=[early_stopping],
    )

    trainer.train()

    # Evaluate and save best model globally
    eval_metrics = trainer.evaluate()
    current_f1 = eval_metrics["eval_f1"]

    if current_f1 > best_f1:
        best_f1 = current_f1
        trainer.save_model("./best_model")
        trainer.save_model(GDRIVE_DIR)
        printer(f"New best model saved with F1: {best_f1:.4f}")
    return current_f1

Тренировка модели

In [ ]:
study = optuna.create_study(study_name='test_optuna', direction='maximize')
study.optimize(func=objective, n_trials=5)

[I 2025-05-11 17:27:52,177] A new study created in memory with name: test_optuna
<ipython-input-18-bf58cffe5d0c>:17: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  learning_rate=trial.suggest_loguniform('learning_rate', low=1e-5, high=5e-4),
<ipython-input-18-bf58cffe5d0c>:18: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  weight_decay=trial.suggest_loguniform('weight_decay', 1e-3, 0.1),
The speedups for torchdynamo mostly come with GPU Ampere or higher and which is not detected here.
Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_t

Epoch,Training Loss,Validation Loss,Precision,Recall,F1,Accuracy
1,0.040500,0.060844,0.928359,0.931132,0.929743,0.980729
2,0.028600,0.049223,0.942584,0.947327,0.944950,0.984621
3,0.039900,0.045600,0.944822,0.961164,0.952923,0.986750
4,0.035000,0.048347,0.955389,0.954638,0.955014,0.987271
5,0.013000,0.048216,0.955758,0.962972,0.959352,0.988369
6,0.010200,0.051632,0.954207,0.966509,0.960319,0.988568


[I 2025-05-11 18:08:17,611] Trial 0 finished with value: 0.9603187002030932 and parameters: {'learning_rate': 0.00010983291256345037, 'weight_decay': 0.0769747453319421, 'num_train_epochs': 6}. Best is trial 0 with value: 0.9603187002030932.
<ipython-input-18-bf58cffe5d0c>:17: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  learning_rate=trial.suggest_loguniform('learning_rate', low=1e-5, high=5e-4),
<ipython-input-18-bf58cffe5d0c>:18: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  weight_decay=trial.suggest_loguniform('weight_decay', 1e-3, 0.1),
The speedups for torchdynamo mostly come with GPU Ampere or higher and which is not detected here.
Using the `WANDB_DISABLED` en

******************************************************************************************************************************************************

New best model saved with F1: 0.9603

******************************************************************************************************************************************************



<ipython-input-18-bf58cffe5d0c>:39: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


...

[I 2025-05-11 19:58:19,121] Trial 3 finished with value: 0.9612900700614506 and parameters: {'learning_rate': 0.00019863319858102557, 'weight_decay': 0.0861746314370445, 'num_train_epochs': 7}. Best is trial 3 with value: 0.9612900700614506.
<ipython-input-18-bf58cffe5d0c>:17: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  learning_rate=trial.suggest_loguniform('learning_rate', low=1e-5, high=5e-4),
<ipython-input-18-bf58cffe5d0c>:18: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  weight_decay=trial.suggest_loguniform('weight_decay', 1e-3, 0.1),
The speedups for torchdynamo mostly come with GPU Ampere or higher and which is not detected here.
Using the `WANDB_DISABLED` en

******************************************************************************************************************************************************

New best model saved with F1: 0.9613

******************************************************************************************************************************************************



<ipython-input-18-bf58cffe5d0c>:39: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


Epoch,Training Loss,Validation Loss,Precision,Recall,F1,Accuracy
1,0.006700,0.078855,0.945065,0.950786,0.947917,0.985187
2,0.018900,0.064634,0.940084,0.963365,0.951582,0.986628
3,0.009400,0.065708,0.953962,0.964387,0.959146,0.988347
4,0.003100,0.066889,0.958467,0.963365,0.960910,0.988746
5,0.003400,0.070513,0.956603,0.963522,0.960050,0.988624


[I 2025-05-11 20:28:16,015] Trial 4 finished with value: 0.960909625563615 and parameters: {'learning_rate': 0.0001751577899781173, 'weight_decay': 0.0069794331930515575, 'num_train_epochs': 5}. Best is trial 3 with value: 0.9612900700614506.


In [ ]:
best_trial = study.best_trial
printer(f"Final best F1: {best_trial.value}")
printer(f"Final params: {best_trial.params}")

******************************************************************************************************************************************************

Final best F1: 0.9612900700614506

******************************************************************************************************************************************************

******************************************************************************************************************************************************

Final params: {'learning_rate': 0.00019863319858102557, 'weight_decay': 0.0861746314370445, 'num_train_epochs': 7}

******************************************************************************************************************************************************



In [ ]:
best_trial = study.best_trial
printer(f"Final best F1: {best_trial.value}")
printer(f"Final params: {best_trial.params}")

******************************************************************************************************************************************************

Final best F1: 0.0

******************************************************************************************************************************************************

******************************************************************************************************************************************************

Final params: {'learning_rate': 0.00027595723461451484, 'weight_decay': 0.005073740110800019, 'num_train_epochs': 2}

******************************************************************************************************************************************************



In [ ]:
optuna.visualization.plot_param_importances(study).show()

In [ ]:
optuna.visualization.plot_param_importances(study).show()

# Model evaluation

Загрузка сохраненной модели и проверка на описаниях, которых нет в датасете

In [ ]:
best_model_path = '/content/drive/MyDrive/vkr_pandan/nuner_230425/best_model'
final_tokenizer = AutoTokenizer.from_pretrained(best_model_path, use_fast=True, add_prefix_space=True)
final_model = AutoModelForTokenClassification.from_pretrained(best_model_path)


In [ ]:
s = '''
All versions of package git-clone are vulnerable to Command Injection due to insecure usage of the --upload-pack feature of git.
'''

In [ ]:
token_classifier = pipeline(
    "token-classification", model=final_model, aggregation_strategy="first", tokenizer=final_tokenizer
)
res = token_classifier(s)
for i, r in enumerate(res):
    # print('Entity: '+ r['entity_group'] + '   Word: ' + r['word'])
    print('Entity: '+ r['entity_group'] + '   Word: ' + r['word'] + '. Probs:  ' + str(round(r['score'], 4)))

Device set to use cpu


Entity: product   Word:  git-clone. Probs:  0.7367
Entity: product   Word:  git.
. Probs:  0.8289


In [ ]:
res

[{'entity_group': 'vendor',
  'score': np.float32(0.5412197),
  'word': ' ',
  'start': 0,
  'end': 0},
 {'entity_group': 'vendor',
  'score': np.float32(0.7811903),
  'word': '\nIBM',
  'start': 0,
  'end': 4},
 {'entity_group': 'product',
  'score': np.float32(0.9995137),
  'word': ' InfoSphere Information Server',
  'start': 5,
  'end': 34},
 {'entity_group': 'version',
  'score': np.float32(0.9998714),
  'word': ' 11.7',
  'start': 35,
  'end': 39}]

In [ ]:
test_tokens = tokenized_ds['test'].select(range(10))['tokens']
test_labels = tokenized_ds['test'].select(range(10))['labels']

### working with 1 string

In [ ]:
encoded_input = final_tokenizer(' '.join(test_tokens[0]), truncation=True, padding=True, return_tensors='pt')
output = final_model(**encoded_input)
res = output.logits.argmax(dim=-1)[0]
res

In [ ]:
print(len(res))
print(len(test_labels[0]))

### Eval NuNER

In [ ]:
nuner_best_model_path = '/content/drive/MyDrive/vkr_pandan/nuner_110525/best_model/'
nuner_tokenizer = AutoTokenizer.from_pretrained(nuner_best_model_path, use_fast=True, add_prefix_space=True)
nuner_model = AutoModelForTokenClassification.from_pretrained(nuner_best_model_path)


In [ ]:
nuner_tokenized_ds_test = ds_test.map(lambda x: tokenize_and_align_labels(x, nuner_tokenizer), batched=True)

Map:   0%|          | 0/1532 [00:00<?, ? examples/s]

In [ ]:
from sklearn.metrics import confusion_matrix, classification_report

def compute_metrics_custom_evaluate(p):
    lbls_in_dataset = [
        'O',
        'B-product',
        'I-product',
        'B-vendor',
        'I-vendor',
        'B-version',
        'I-version',
    ]
    label2id = {v:i for i, v in enumerate(lbls_in_dataset)}
    id2label = {i:v for i, v in enumerate(lbls_in_dataset)}

    labels_list = list(id2label.values())
    predictions, labels = p
    predictions = np.argmax(predictions, axis=2)

    # Уберем I-vendor, т.к. эта сущность представлена слабо
    true_predictions = [
        [labels_list[p] for (p, l) in zip(prediction, label) if l != -100 and  l != 4]
        for prediction, label in zip(predictions, labels)
    ]
    true_labels = [
        [labels_list[l] for (p, l) in zip(prediction, label) if l != -100 and l != 4]
        for prediction, label in zip(predictions, labels)
    ]
    # Classification report
    flat_true_labels = [l for sublist in true_labels for l in sublist if l != 4]
    flat_predictions = [p for sublist in true_predictions for p in sublist if p != 4]


    label_list = nuner_tokenized_ds_test.features["ner_tags"].feature.names[1:]
    label_list.remove("I-vendor")

    report = classification_report(
        flat_true_labels,
        flat_predictions,
        labels=label_list,
        output_dict=True
    )

    # Confusion matrix
    cm = confusion_matrix(flat_true_labels, flat_predictions, labels=label_list)

    return {
        "classification_report": report,
        "confusion_matrix": cm
    }

In [ ]:
data_collator = DataCollatorForTokenClassification(nuner_tokenizer)
trainer = Trainer(
    model=nuner_model,
    data_collator=data_collator,
    compute_metrics=compute_metrics_custom_evaluate,
)

# 9. Run evaluation
results = trainer.predict(nuner_tokenized_ds_test)
# 10. Print results
report = results.metrics["test_classification_report"]
cm = results.metrics["test_confusion_matrix"]

print("Per-class metrics:")
report_df = pd.DataFrame(report).transpose()
print(report_df[["precision", "recall", "f1-score", "support"]].iloc[:-3])


Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).


Per-class metrics:
           precision    recall  f1-score  support
B-product   0.933447  0.942562  0.937982   3482.0
I-product   0.903664  0.955524  0.928870   1394.0
B-vendor    0.938382  0.934911  0.936643   1352.0
B-version   0.987806  0.988147  0.987977   8690.0
I-version   0.997386  0.994668  0.996025   8440.0


In [ ]:
report_df.round(4)

,precision,recall,f1-score,support
B-product,0.9334,0.9426,0.9380,3482.0
I-product,0.9037,0.9555,0.9289,1394.0
B-vendor,0.9384,0.9349,0.9366,1352.0
B-version,0.9878,0.9881,0.9880,8690.0
I-version,0.9974,0.9947,0.9960,8440.0
micro avg,0.9750,0.9787,0.9768,23358.0
macro avg,0.9521,0.9632,0.9575,23358.0
weighted avg,0.9753,0.9787,0.9769,23358.0


In [ ]:
print(report_df.round(4).to_markdown())

|              |   precision |   recall |   f1-score |   support |
|:-------------|------------:|---------:|-----------:|----------:|
| B-product    |      0.9334 |   0.9426 |     0.938  |      3482 |
| I-product    |      0.9037 |   0.9555 |     0.9289 |      1394 |
| B-vendor     |      0.9384 |   0.9349 |     0.9366 |      1352 |
| B-version    |      0.9878 |   0.9881 |     0.988  |      8690 |
| I-version    |      0.9974 |   0.9947 |     0.996  |      8440 |
| micro avg    |      0.975  |   0.9787 |     0.9768 |     23358 |
| macro avg    |      0.9521 |   0.9632 |     0.9575 |     23358 |
| weighted avg |      0.9753 |   0.9787 |     0.9769 |     23358 |


|              |   precision |   recall |   f1-score |   support |
|:-------------|------------:|---------:|-----------:|----------:|
| B-product    |      0.9334 |   0.9426 |     0.938  |      3482 |
| I-product    |      0.9037 |   0.9555 |     0.9289 |      1394 |
| B-vendor     |      0.9384 |   0.9349 |     0.9366 |      1352 |
| B-version    |      0.9878 |   0.9881 |     0.988  |      8690 |
| I-version    |      0.9974 |   0.9947 |     0.996  |      8440 |
| micro avg    |      0.975  |   0.9787 |     0.9768 |     23358 |
| macro avg    |      0.9521 |   0.9632 |     **0.9575** |     23358 |
| weighted avg |      0.9753 |   0.9787 |     0.9769 |     23358 |



### Eval BERT

In [ ]:
bert_best_model_path = '/content/drive/MyDrive/vkr_pandan/bert_cased_kaggle_110525'
bert_tokenizer = AutoTokenizer.from_pretrained(bert_best_model_path, use_fast=True, add_prefix_space=True)
bert_model = AutoModelForTokenClassification.from_pretrained(bert_best_model_path)


In [ ]:
bert_tokenized_ds_test = ds_test.map(lambda x: tokenize_and_align_labels(x, bert_tokenizer), batched=True)

Map:   0%|          | 0/1532 [00:00<?, ? examples/s]

In [ ]:
from sklearn.metrics import confusion_matrix, classification_report

def compute_metrics_custom_evaluate(p):
    lbls_in_dataset = [
        'O',
        'B-product',
        'I-product',
        'B-vendor',
        'I-vendor',
        'B-version',
        'I-version',
    ]
    label2id = {v:i for i, v in enumerate(lbls_in_dataset)}
    id2label = {i:v for i, v in enumerate(lbls_in_dataset)}

    labels_list = list(id2label.values())
    predictions, labels = p
    predictions = np.argmax(predictions, axis=2)

    # Уберем I-vendor, т.к. эта сущность представлена слабо
    true_predictions = [
        [labels_list[p] for (p, l) in zip(prediction, label) if l != -100 and  l != 4]
        for prediction, label in zip(predictions, labels)
    ]
    true_labels = [
        [labels_list[l] for (p, l) in zip(prediction, label) if l != -100 and l != 4]
        for prediction, label in zip(predictions, labels)
    ]
    # Classification report
    flat_true_labels = [l for sublist in true_labels for l in sublist if l != 4]
    flat_predictions = [p for sublist in true_predictions for p in sublist if p != 4]


    label_list = bert_tokenized_ds_test.features["ner_tags"].feature.names[1:]
    label_list.remove("I-vendor")

    report = classification_report(
        flat_true_labels,
        flat_predictions,
        labels=label_list,
        output_dict=True
    )

    # Confusion matrix
    cm = confusion_matrix(flat_true_labels, flat_predictions, labels=label_list)

    return {
        "classification_report": report,
        "confusion_matrix": cm
    }

In [ ]:
data_collator = DataCollatorForTokenClassification(bert_tokenizer)
trainer = Trainer(
    model=bert_model,
    data_collator=data_collator,
    compute_metrics=compute_metrics_custom_evaluate,
)

# 9. Run evaluation
results = trainer.predict(bert_tokenized_ds_test)
# 10. Print results
report = results.metrics["test_classification_report"]
cm = results.metrics["test_confusion_matrix"]

print("Per-class metrics:")
report_df_bert = pd.DataFrame(report).transpose()
print(report_df_bert[["precision", "recall", "f1-score", "support"]].iloc[:-3])


Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).


Per-class metrics:
           precision    recall  f1-score  support
B-product   0.928752  0.934645  0.931689   4728.0
I-product   0.882449  0.939446  0.910056   1734.0
B-vendor    0.932308  0.915961  0.924062   1654.0
B-version   0.987852  0.981106  0.984468   8786.0
I-version   0.997547  0.994642  0.996092   8585.0


In [ ]:
report_df_bert.round(4)

,precision,recall,f1-score,support
B-product,0.9288,0.9346,0.9317,4728.0
I-product,0.8824,0.9394,0.9101,1734.0
B-vendor,0.9323,0.9160,0.9241,1654.0
B-version,0.9879,0.9811,0.9845,8786.0
I-version,0.9975,0.9946,0.9961,8585.0
micro avg,0.9689,0.9700,0.9695,25487.0
macro avg,0.9458,0.9532,0.9493,25487.0
weighted avg,0.9694,0.9700,0.9696,25487.0


In [ ]:
print(report_df_bert.round(4).to_markdown())

|              |   precision |   recall |   f1-score |   support |
|:-------------|------------:|---------:|-----------:|----------:|
| B-product    |      0.9288 |   0.9346 |     0.9317 |      4728 |
| I-product    |      0.8824 |   0.9394 |     0.9101 |      1734 |
| B-vendor     |      0.9323 |   0.916  |     0.9241 |      1654 |
| B-version    |      0.9879 |   0.9811 |     0.9845 |      8786 |
| I-version    |      0.9975 |   0.9946 |     0.9961 |      8585 |
| micro avg    |      0.9689 |   0.97   |     0.9695 |     25487 |
| macro avg    |      0.9458 |   0.9532 |     0.9493 |     25487 |
| weighted avg |      0.9694 |   0.97   |     0.9696 |     25487 |


|              |   precision |   recall |   f1-score |   support |
|:-------------|------------:|---------:|-----------:|----------:|
| B-product    |      0.9288 |   0.9346 |     0.9317 |      4728 |
| I-product    |      0.8824 |   0.9394 |     0.9101 |      1734 |
| B-vendor     |      0.9323 |   0.916  |     0.9241 |      1654 |
| B-version    |      0.9879 |   0.9811 |     0.9845 |      8786 |
| I-version    |      0.9975 |   0.9946 |     0.9961 |      8585 |
| micro avg    |      0.9689 |   0.97   |     0.9695 |     25487 |
| macro avg    |      0.9458 |   0.9532 |     **0.9493** |     25487 |
| weighted avg |      0.9694 |   0.97   |     0.9696 |     25487 |



# Get NER from random CVE descriptions

In [ ]:
df_test = pd.read_csv('/content/df_100_not_in_stucco_v2.csv')

In [ ]:
df_test = df_test.iloc[:10]

In [ ]:
s = """
Auth. (contributor+) Stored Cross-Site Scripting (XSS) vulnerability in Julien Berthelot / MPEmbed WP Matterport Shortcode plugin <= 2.1.4 versions.
"""

token_classifier = pipeline(
    "token-classification", model=bert_model, aggregation_strategy="first", tokenizer=bert_tokenizer
)
res = token_classifier(s)
for i, r in enumerate(res):
    # print('Entity: '+ r['entity_group'] + '   Word: ' + r['word'])
    print('Entity: '+ r['entity_group'] + '   Word: ' + r['word'] + '   Prob: ' + str(r['score']))

Device set to use cpu


Entity: product   Word: wp   Prob: 0.56172967
Entity: version   Word: <   Prob: 0.99953616
Entity: version   Word: = 2. 1. 4   Prob: 0.998993


In [ ]:
vendor = []
product = []
version = []
vendor_probs = []
product_probs = []
version_probs = []

for ner_item in res:
    if ner_item['entity_group'] == 'vendor':
        vendor.append(str.lower(ner_item['word'].strip()))
        vendor_probs.append(str.lower(str(ner_item['score'])))
    elif ner_item['entity_group'] == 'product':
        product.append(str.lower(str(ner_item['word'].strip())))
        product_probs.append(str.lower(str(ner_item['score'])))
    elif ner_item['entity_group'] == 'version':
        version.append(str.lower(str(ner_item['word'].strip())))
        version_probs.append(str.lower(str(ner_item['score'])))

In [ ]:
product

['io', '.', 'ratpack', ':', 'ratpack', '-', 'core']

In [ ]:
s = """
In Progress MOVEit Transfer versions released before 2022.0.11 (14.0.11), 2022.1.12 (14.1.12), 2023.0.9 (15.0.9), 2023.1.4 (15.1.4), a logging bypass vulnerability has been discovered.  An authenticated user could manipulate a request to bypass the logging mechanism within the web application which results in user activity not being logged properly.
"""
token_classifier = pipeline(
    "token-classification", model=bert_model, aggregation_strategy="first", tokenizer=bert_tokenizer
)
res = token_classifier(s)
for i, r in enumerate(res):
    # print('Entity: '+ r['entity_group'] + '   Word: ' + r['word'])
    print('Entity: '+ r['entity_group'] + '   Word: ' + r['word'] + '   Prob: ' + str(r['score']))
print('*'*100)
token_classifier = pipeline(
    "token-classification", model=nuner_model, aggregation_strategy="first", tokenizer=nuner_tokenizer
)
res = token_classifier(s)
for i, r in enumerate(res):
    # print('Entity: '+ r['entity_group'] + '   Word: ' + r['word'])
    print('Entity: '+ r['entity_group'] + '   Word: ' + r['word'] + '   Prob: ' + str(r['score']))

Device set to use cuda:0
Device set to use cuda:0


Entity: product   Word: Progress MOVEit Transfer   Prob: 0.7903766
Entity: version   Word: before 2022. 0. 11 ( 14. 0. 11 ),   Prob: 0.954353
Entity: version   Word: 2023   Prob: 0.569417
Entity: version   Word: 0   Prob: 0.5783962
Entity: version   Word: 9   Prob: 0.53015035
Entity: version   Word: 2023   Prob: 0.88098097
Entity: version   Word: .   Prob: 0.662273
Entity: version   Word: 1   Prob: 0.75145066
Entity: version   Word: 4   Prob: 0.5396657
****************************************************************************************************
Entity: vendor   Word:  Progress   Prob: 0.9793504
Entity: product   Word:  MOVEit Transfer   Prob: 0.9827735
Entity: version   Word:  before 2022.0.11   Prob: 0.9997338
Entity: version   Word:  2022.1.12   Prob: 0.99994123
Entity: version   Word:  2023.0.9   Prob: 0.9999453
Entity: version   Word:  2023.1.4   Prob: 0.99995375


In [ ]:
# def extract_ners(cve, tokenizer=final_tokenizer, model=final_model):
#     token_classifier = pipeline(
#         "token-classification", model=final_model, aggregation_strategy="first", tokenizer=final_tokenizer
#     )
#     result = token_classifier(cve)
#     return result

In [ ]:
def extract_ners(cve, tokenizer=bert_tokenizer, model=bert_model):
    token_classifier = pipeline(
        "token-classification", model=bert_model, aggregation_strategy="first", tokenizer=bert_tokenizer
    )
    result = token_classifier(cve)
    vendor = []
    product = []
    version = []
    vendor_probs = []
    product_probs = []
    version_probs = []

    for ner_item in result:
        if ner_item['entity_group'] == 'vendor':
            vendor.append(str.lower(ner_item['word'].strip()))
            vendor_probs.append(str.lower(str(ner_item['score'])))
        elif ner_item['entity_group'] == 'product':
            product.append(str.lower(str(ner_item['word'].strip())))
            product_probs.append(str.lower(str(ner_item['score'])))
        elif ner_item['entity_group'] == 'version':
            version.append(str.lower(str(ner_item['word'].strip())))
            version_probs.append(str.lower(str(ner_item['score'])))
    return {'ners': [vendor, product, version], 'scores': [vendor_probs, product_probs, version_probs]}

In [ ]:
df_test[['ners_list', 'scores_list']] = df_test['descr'].apply(lambda x: extract_ners(x)).apply(pd.Series)
df_test['vendor_ner'], df_test['product_ner'], df_test['version_ner'] = zip(*df_test['ners_list'])
df_test['vendor_score_ner'], df_test['product_score_ner'], df_test['version_score_ner'] = zip(*df_test['scores_list'])

Device set to use cpu
Device set to use cpu
Device set to use cpu
Device set to use cpu
Device set to use cpu
Device set to use cpu
Device set to use cpu
Device set to use cpu
Device set to use cpu
Device set to use cpu


In [ ]:
df_test

,cve_id,cpe_id_pk,vendor,product,version,descr,initial_cpe,ners_list,scores_list,vendor_ner,product_ner,version_ner,vendor_score_ner,product_score_ner,version_score_ner
0,CVE-2023-0878,294697,nuxt,nuxt,1.0.0,Cross-site Scripting (XSS) - Generic in GitHub repository nuxt/framework prior to 3.2.1.,cpe:2.3:a:nuxt:nuxt:1.0.0:rc8:*:*:*:*:*:*,"[[], [], [prior to 3. 2. 1]]","[[], [], [0.9997641]]",[],[],[prior to 3. 2. 1],[],[],[0.9997641]
1,CVE-2008-0444,273643,elog,elog,2.2.0,Cross-site scripting (XSS) vulnerability in Electronic Logbook (ELOG) before 2.7.0 allows remote...,cpe:2.3:a:elog:elog:2.2.0:*:*:*:*:*:*:*,"[[], [electronic logbook, elog], [before 2. 7. 0]]","[[], [0.7493671, 0.9961223], [0.99990225]]",[],"[electronic logbook, elog]",[before 2. 7. 0],[],"[0.7493671, 0.9961223]",[0.99990225]
2,CVE-2020-14518,460900,philips,dreammapper,2.17.1,"Philips DreamMapper, Version 2.24 and prior. Information written to log files can give guidance ...",cpe:2.3:a:philips:dreammapper:2.17.1:*:*:*:*:iphone_os:*:*,"[[philips], [dreammapper], [2, ., 24 and prior]]","[[0.9969893], [0.9989706], [0.86764824, 0.9387206, 0.99204534]]",[philips],[dreammapper],"[2, ., 24 and prior]",[0.9969893],[0.9989706],"[0.86764824, 0.9387206, 0.99204534]"
3,CVE-2022-27858,141525,activity_log_project,activity_log,2.2.8,CSV Injection vulnerability in Activity Log Team Activity Log <= 2.8.3 on WordPress.\n\n,cpe:2.3:a:activity_log_project:activity_log:2.2.8:*:*:*:*:wordpress:*:*,"[[], [activity log team activity log], [<, = 2. 8. 3]]","[[], [0.9995812], [0.999785, 0.99988294]]",[],[activity log team activity log],"[<, = 2. 8. 3]",[],[0.9995812],"[0.999785, 0.99988294]"
4,CVE-2022-4725,214060,amazon,aws_software_development_kit,2.2.4,A vulnerability was found in AWS SDK 2.59.0. It has been rated as critical. This issue affects t...,cpe:2.3:a:amazon:aws_software_development_kit:2.2.4:*:*:*:*:android:*:*,"[[], [aws sdk], [2, ., 59, ., 0, upgrading to version 2. 59. 1]]","[[], [0.9997598], [0.9998342, 0.9998173, 0.9998312, 0.99982834, 0.9998141, 0.99893343]]",[],[aws sdk],"[2, ., 59, ., 0, upgrading to version 2. 59. 1]",[],[0.9997598],"[0.9998342, 0.9998173, 0.9998312, 0.99982834, 0.9998141, 0.99893343]"
5,CVE-2021-32691,625325,apollosapp,data-connector-rock,1.2.5,Apollos Apps is an open source platform for launching church-related apps. In Apollos Apps versi...,cpe:2.3:a:apollosapp:data-connector-rock:1.2.5:-:*:*:*:node.js:*:*,"[[], [apollos apps, apollos apps], [prior to 2. 20. 0, 2, ., 20, ., 0]]","[[], [0.98203385, 0.9907462], [0.9998293, 0.94735706, 0.8981481, 0.98013914, 0.96440566, 0.94913...",[],"[apollos apps, apollos apps]","[prior to 2. 20. 0, 2, ., 20, ., 0]",[],"[0.98203385, 0.9907462]","[0.9998293, 0.94735706, 0.8981481, 0.98013914, 0.96440566, 0.9491303]"
6,CVE-2016-4572,435704,cloudera,cdh,5.5.1,"In Cloudera CDH before 5.7.1, Impala REVOKE ALL ON SERVER commands do not revoke all privileges.",cpe:2.3:a:cloudera:cdh:5.5.1:*:*:*:*:*:*:*,"[[], [cloudera cdh], [before 5. 7. 1]]","[[], [0.9996662], [0.9999276]]",[],[cloudera cdh],[before 5. 7. 1],[],[0.9996662],[0.9999276]
7,CVE-2023-31441,341885,ncia,advisor_network,2.2.0,"In NATO Communications and Information Agency anet (aka Advisor Network) through 3.3.0, an attac...",cpe:2.3:a:ncia:advisor_network:2.2.0:*:*:*:*:*:*:*,"[[nato], [], [through 3. 3. 0]]","[[0.9864487], [], [0.9999071]]",[nato],[],[through 3. 3. 0],[0.9864487],[],[0.9999071]
8,CVE-2023-42261,372961,opensecurity,mobile_security_framework,0.9.4.1,Mobile Security Framework (MobSF) <=v3.7.8 Beta is vulnerable to Insecure Permissions. NOTE: the...,cpe:2.3:a:opensecurity:mobile_security_framework:0.9.4.1:*:*:*:*:*:*:*,"[[], [mobile security, mobsf], [<, = v3. 7. 8]]","[[], [0.87309456, 0.98690516], [0.99979204, 0.9993396]]",[],"[mobile security, mobsf]","[<, = v3. 7. 8]",[],"[0.87309456, 0.98690516]","[0.99979204, 0.9993396]"
9,CVE-2022-36532,175865,bolt,bolt_cms,3.0.2,Bolt CMS contains a vulnerability in version 5.1.12 and below that allows a

In [ ]:
df_test[['ners_list2', 'scores_list2']] = df_test['descr'].apply(lambda x: extract_ners(x, model=final_model2, tokenizer=final_tokenizer2)).apply(pd.Series)
df_test['vendor_ner2'], df_test['product_ner2'], df_test['version_ner2'] = zip(*df_test['ners_list2'])
df_test['vendor_score_ner2'], df_test['product_score_ner2'], df_test['version_score_ner2'] = zip(*df_test['scores_list2'])

Device set to use cpu
/usr/local/lib/python3.11/dist-packages/transformers/pipelines/token_classification.py:398: UserWarning: Tokenizer does not support real words, using fallback heuristic
  warnings.warn(
Device set to use cpu
Device set to use cpu
Device set to use cpu
Device set to use cpu
Device set to use cpu
Device set to use cpu
Device set to use cpu
Device set to use cpu
Device set to use cpu
Device set to use cpu
Device set to use cpu
Device set to use cpu
Device set to use cpu
Device set to use cpu
Device set to use cpu
Device set to use cpu
Device set to use cpu
Device set to use cpu
Device set to use cpu
Device set to use cpu
Device set to use cpu
Device set to use cpu
Device set to use cpu
Device set to use cpu
Device set to use cpu
Device set to use cpu
Device set to use cpu
Device set to use cpu
Device set to use cpu
Device set to use cpu
Device set to use cpu
Device set to use cpu
Device set to use cpu
Device set to use cpu
Device set to use cpu
Device set to use cpu


In [ ]:
df_test[['product_ner', 'product_ner2']]

,product_ner,product_ner2
0,[],[]
1,"[jenkins maven, choicelistprovider]","[jenkins maven, choicelistprovider]"
2,"[pedro lineu orso, chetcpasswd]","[pedro lineu orso, chetcpasswd]"
3,[],[]
4,"[wp go maps, wp google maps)]","[wp go maps, wp google maps)]"
5,[],[]
6,"[zoom client for, zoom meeting url]","[zoom client for, zoom meeting url]"
7,[],[]
8,[],[]
9,[scylladb],[scylladb]


In [ ]:
df_test[['product', 'product_ner']].sample(5)

In [ ]:
df_test[['version', 'version_ner', 'initial_cpe', 'descr']]

In [ ]:
df_test.to_excel('initial_cpe_190425.xlsx', index=False)

## Если выделилось несколько продуктов или вендоров

Но на самом деле это повторение одинаковых, то оставим только одно

## Deduplicate using lcs function (old)

In [ ]:
A = 'spring security'
B = 'spring'
print(pylcs.lcs_sequence_length(A, B) / len(A))
print(pylcs.lcs_sequence_length(A, B) / len(B))

0.4
1.0


In [ ]:
def deduplicate_entities_lcs(ners: list):
    '''
    If there are multiple almost identical enities in the description, deduplicate them.
    '''
    if len(ners) == 1:
        return ners[0]
    elif not ners:
        return ''
    first_ner = ners[0]
    print(first_ner)
    for ner in ners[1:]:
        lcs_A = pylcs.lcs_sequence_length(first_ner, ner) / len(first_ner)
        lcs_B = pylcs.lcs_sequence_length(first_ner, ner) / len(ner)
        if lcs_A > 0.9 or lcs_B > 0.9:
            return max(first_ner, ner)
        else:
            return max

In [ ]:
print(deduplicate_entities(['spring security', 'spring']))
print(deduplicate_entities(['git']))
print(deduplicate_entities([' iterm2' , 'iterm2']))

spring security
git
iterm2


In [ ]:
print(df_test['ners_list'].iloc[1][1])
print(deduplicate_entities(df_test['ners_list'].iloc[1][1]))

[' jenkins maven', ' choicelistprovider']
 jenkins maven
 jenkins maven


In [ ]:
df_test['dedup_vendor'] = df_test.apply(lambda x: deduplicate_entities(x['ners_list'][0]), axis=1)
df_test['dedup_product'] = df_test.apply(lambda x: deduplicate_entities(x['ners_list'][1]), axis=1)

In [ ]:
df_test.iloc[:10]

,cve_id_pk,cve_id,descr_id,cve_id_fk,descr,cve_id_fk.1,cpe_id_fk,config_id,cpe_id_pk,cpe_version,part,vendor,product,version,update,edition,sw_edition,target_sw,target_hw,language,other,initial_cpe,cve_year,ners_list,scores_list,vendor_ner,product_ner,version_ner,vendor_score_ner,product_score_ner,version_score_ner,dedup_product,dedup_product_score,dedup_vendor,dedup_vendor_score
0,68274,CVE-2023-49619,19660,68274,Concurrent Execution using Shared Resource with Improper Synchronization ('Race Condition') vuln...,68274,394223,969097,394223,2.3,a,apache,answer,0.5.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,cpe:2.3:a:apache:answer:0.5.0:*:*:*:*:*:*:*,2023,"[[apache], [], [through 1.2.0.\n\nunder]]","[[0.99848026], [], [0.99976075]]",[apache],[],[through 1.2.0.\n\nunder],[0.99848026],[],[0.99976075],[],[],[ apache],[0.99848026]
1,63977,CVE-2023-40347,15363,63977,Jenkins Maven Artifact ChoiceListProvider (Nexus) Plugin 1.14 and earlier does not set the appro...,63977,368852,863882,368852,2.3,a,jenkins,maven_artifact_choicelistprovider_\(nexus\),0.0.6,NaN,NaN,NaN,NaN,jenkins,NaN,NaN,cpe:2.3:a:jenkins:maven_artifact_choicelistprovider_\(nexus\):0.0.6:*:*:*:*:jenkins:*:*,2023,"[[], [jenkins maven, choicelistprovider], [artifact, 1.14 and earlier]]","[[], [0.68806636, 0.9807293], [0.37263352, 0.99993324]]",[],"[jenkins maven, choicelistprovider]","[artifact, 1.14 and earlier]",[],"[0.68806636, 0.9807293]","[0.37263352, 0.99993324]",[ choicelistprovider],[0.9807293],[],[]
2,176167,CVE-2002-2220,85401,176167,"Buffer overflow in Pedro Lineu Orso chetcpasswd before 1.12, when configured for access from 0.0...",176167,731340,69942,731340,2.3,a,chetcpasswd,chetcpasswd,1.12,NaN,NaN,NaN,NaN,NaN,NaN,NaN,cpe:2.3:a:chetcpasswd:chetcpasswd:1.12:*:*:*:*:*:*:*,2002,"[[], [pedro lineu orso, chetcpasswd], [before 1.12,, 0.0.0.0,]]","[[], [0.4913182, 0.7372176], [0.9998815, 0.99950683]]",[],"[pedro lineu orso, chetcpasswd]","[before 1.12,, 0.0.0.0,]",[],"[0.4913182, 0.7372176]","[0.9998815, 0.99950683]",[ chetcpasswd],[0.7372176],[],[]
3,71348,CVE-2016-0851,102506,71348,Advantech WebAccess before 8.1 allows remote attackers to cause a denial of service (out-of-boun...,71348,417375,19280,417375,2.3,a,advantech,webaccess,7.0-2012.12.10,NaN,NaN,NaN,NaN,NaN,NaN,NaN,cpe:2.3:a:advantech:webaccess:7.0-2012.12.10:*:*:*:*:*:*:*,2016,"[[advantech], [], [before 8.1]]","[[0.9892657], [], [0.99993247]]",[advantech],[],[before 8.1],[0.9892657],[],[0.99993247],[],[],[ advantech],[0.9892657]
4,34073,CVE-2022-47595,175843,34073,Improper Limitation of a Pathname to a Restricted Directory ('Path Traversal') vulnerability in ...,34073,216534,486886,216534,2.3,a,codecabin,wp_go_maps,9.0.12,NaN,NaN,NaN,pro,wordpress,NaN,NaN,cpe:2.3:a:codecabin:wp_go_maps:9.0.12:*:*:*:pro:wordpress:*:*,2022,"[[], [wp go maps, wp google maps)], []]","[[], [0.9817508, 0.9798541], []]",[],"[wp go maps, wp google maps)]",[],[],"[0.9817508, 0.9798541]",[],[ wp go maps],[0.9817508],[],[]
5,89150,CVE-2020-3622,148375,89150,u'Channel name string which has been read from shared memory is potentially subjected to string ...,89150,487329,247753,487329,2.3,h,qualcomm,sxr1130,-,NaN,NaN,NaN,NaN,NaN,NaN,NaN,cpe:2.3:h:qualcomm:sxr1130:-:*:*:*:*:*:*:*,2020,"[[snapdragon, snapdragon, snapdragon, snapdragon, snapdragon, snapdragon, snapdragon, snapdragon...","[[0.83081603, 0.8796247, 0.7142722, 0.86553067, 0.84401894, 0.90823996, 0.71996105, 0.8062775], ...","[snapdragon, snapdragon, snapdragon, snapdragon, snapdragon, snapdragon, snapdragon, snapdragon]",[],[],"[0.83081603, 0.8796247, 0.7142722, 0.86553067, 0.84401894, 0.90823996, 0.71996105, 0.8062775]",[],[],[],[],[ snapdragon],[0.90823996]
6,23983,CVE-2022-28755,165753,23983,"The Zoom Client for Meetings (for Android, iOS, Linux, macOS, and Windows) before version 5.11.0...",23983,144690,235461,144690,2.3,a,zoom,zoom,3.5.6558.0105,NaN,NaN,NaN,NaN,macos,NaN,NaN,cpe:2.3:a:zoom:zoom:3.5.6558.0105:*:*:*:*:macos:*:*,2022,"[[], [zoom client for, zoom meeting url], [5.11.0]]"

## Deduplicate using highest probabilities

Дедупликация по порогу, чтобы оставались несколько вероятных вендоров\продуктов. Проверка как соотносится вендор и продукт на основе информации из БД.

In [ ]:
def deduplicate_using_probs(row, ner_type):
    if ner_type == 'product':
        product_ner = row['product_ner']
        product_score_ner = row['product_score_ner']
        if not product_ner or len(product_ner) == 1:
            return product_ner, product_score_ner
        else:
            max_idx = product_score_ner.index(max(product_score_ner))
            return [[product_ner[max_idx]], [product_score_ner[max_idx]]]
    elif ner_type == 'vendor':
        vendor_ner = row['vendor_ner']
        vendor_score_ner = row['vendor_score_ner']
        if not vendor_ner or len(vendor_ner) == 1:
            return vendor_ner, vendor_score_ner
        else:
            max_idx = vendor_score_ner.index(max(vendor_score_ner))
            return [[vendor_ner[max_idx]], [vendor_score_ner[max_idx]]]



In [ ]:
df_test[['dedup_vendor', 'dedup_vendor_score']] = df_test.apply(lambda x: deduplicate_using_probs(x, 'vendor'), axis=1).apply(pd.Series)

In [ ]:
df_test[['dedup_product', 'dedup_product_score']] = df_test.apply(lambda x: deduplicate_using_probs(x, 'product'), axis=1).apply(pd.Series)

In [ ]:
df_test.head(3)

,cve_id_pk,cve_id,descr_id,cve_id_fk,descr,cve_id_fk.1,cpe_id_fk,config_id,cpe_id_pk,cpe_version,part,vendor,product,version,update,edition,sw_edition,target_sw,target_hw,language,other,initial_cpe,cve_year,ners_list,scores_list,vendor_ner,product_ner,version_ner,vendor_score_ner,product_score_ner,version_score_ner,dedup_vendor,dedup_vendor_score,dedup_product,dedup_product_score
0,68274,CVE-2023-49619,19660,68274,Concurrent Execution using Shared Resource with Improper Synchronization ('Race Condition') vuln...,68274,394223,969097,394223,2.3,a,apache,answer,0.5.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,cpe:2.3:a:apache:answer:0.5.0:*:*:*:*:*:*:*,2023,"[[apache], [], [through 1.2.0.\n\nunder]]","[[0.99848026], [], [0.99976075]]",[apache],[],[through 1.2.0.\n\nunder],[0.99848026],[],[0.99976075],[apache],[0.99848026],[],[]
1,63977,CVE-2023-40347,15363,63977,Jenkins Maven Artifact ChoiceListProvider (Nexus) Plugin 1.14 and earlier does not set the appro...,63977,368852,863882,368852,2.3,a,jenkins,maven_artifact_choicelistprovider_\(nexus\),0.0.6,NaN,NaN,NaN,NaN,jenkins,NaN,NaN,cpe:2.3:a:jenkins:maven_artifact_choicelistprovider_\(nexus\):0.0.6:*:*:*:*:jenkins:*:*,2023,"[[], [jenkins maven, choicelistprovider], [artifact, 1.14 and earlier]]","[[], [0.68806636, 0.9807293], [0.37263352, 0.99993324]]",[],"[jenkins maven, choicelistprovider]","[artifact, 1.14 and earlier]",[],"[0.68806636, 0.9807293]","[0.37263352, 0.99993324]",[],[],[choicelistprovider],[0.9807293]
2,176167,CVE-2002-2220,85401,176167,"Buffer overflow in Pedro Lineu Orso chetcpasswd before 1.12, when configured for access from 0.0...",176167,731340,69942,731340,2.3,a,chetcpasswd,chetcpasswd,1.12,NaN,NaN,NaN,NaN,NaN,NaN,NaN,cpe:2.3:a:chetcpasswd:chetcpasswd:1.12:*:*:*:*:*:*:*,2002,"[[], [pedro lineu orso, chetcpasswd], [before 1.12,, 0.0.0.0,]]","[[], [0.4913182, 0.7372176], [0.9998815, 0.99950683]]",[],"[pedro lineu orso, chetcpasswd]","[before 1.12,, 0.0.0.0,]",[],"[0.4913182, 0.7372176]","[0.9998815, 0.99950683]",[],[],[chetcpasswd],[0.7372176]


In [ ]:
df_test.version_ner.values

array([list(['through 1.2.0.\n\nunder']),
       list(['artifact', '1.14 and earlier']),
       list(['before 1.12,', '0.0.0.0,']), list(['before 8.1']), list([]),
       list([]), list(['5.11.0']), list(['through 0.9.8.']), list([]),
       list([]), list(['before 4.1.14', '4.2.x before 4.2.8']),
       list(['through 3.1.03']),
       list(['v4.0.11.0 and earlier', 'v4.0.13.0 and earlier,']),
       list(['4.1.3']), list(['pro.this', 'through 1.6.11.']),
       list(['2.1 through 3.17s,', '2.0.0 through 5.3.2,']),
       list(['2.4.30 through 2.4.55.\n\nspecial']), list(['6.0', '8.0,']),
       list(['before 2.0.3', '2.1.x before 2.1.3']),
       list(['<=\xa02.1.4']), list(['before 2.16.5']),
       list(['before 7.8.3.']), list(['500', '520', '811']),
       list(['6.41.3/6.42rc27']), list(['0.23.19,', '0.23.19']),
       list(['before 3.24.1']), list(['4.01']), list([]),
       list(['v1.2.0']), list(['through 2021.01']),
       list(['v6.9 and older']), list(['1.3.5 to 1.8.30,'])

In [ ]:
df_test.to_excel('processed_cpe_050425.xlsx', index=False)

## Post-process versions

In [ ]:
def extract_version(matched):
        if matched:
            version = matched.group('version')
            # Normalize separators (replace '-' with '.' if needed)
            version = version.replace('-', '.')
            return version
        return None

def classify_version_string(version_str):
    """
    Generate all versions for expressions containing 'before'.
    Handles two cases:
    1) "before X.Y.Z" - generates all versions up to X.Y.Z
    2) "A.B.x before A.B.C" - generates all patch versions A.B.0 to A.B.(C-1)
    """
    version_str = str.lower(version_str)



    # through, including
    group_name = 'through'
    group_words = ['through', 'earlier', '<=', 'prior', 'up to', 'up to, and including', 'up to and including', 'older']
    for group_wrd in group_words:
        if group_wrd in version_str:
            # More complicated multi version logic
            # '2.1 through 3.17'
            multi_version_pattern = r'(?P<version1>[\dxX]+(?:\s*[.-]\s*[\dxX]+)*)\s*' \
                        r'(?:through|earlier|prior|\<\=|up to)\s*' \
                        r'(?P<version2>[\dxX]+(?:\s*[.-]\s*[\dxX]+)*)'
            multi_match = re.search(multi_version_pattern, version_str, re.IGNORECASE)
            if multi_match:
                version1 = multi_match.group('version1')
                version2 = multi_match.group('version2')
                return [version1, version2], f'{group_name} multi-match'

            pattern = (
                r'(?P<version>[\dxX]+(?:[.-]\s*[\dxX]+)*)'  # Version with digits/x and separators
            )
            matched = re.search(pattern, version_str, re.IGNORECASE)
            return [extract_version(matched)], f'{group_name} group'

    # before, not including
    group_name = 'before'
    group_words = ['before', '<']
    for group_wrd in group_words:
        if group_wrd in version_str:
            # More complicated multi version logic
            # '4.2.x before 4.2.8'
            multi_version_pattern = r'(?P<version1>[\dxX]+(?:\s*[.-]\s*[\dxX]+)*)\s*' \
                        r'(?:before)\s*' \
                        r'(?P<version2>[\dxX]+(?:\s*[.-]\s*[\dxX]+)*)'
            multi_match = re.search(multi_version_pattern, version_str, re.IGNORECASE)
            if multi_match:
                version1 = multi_match.group('version1')
                version2 = multi_match.group('version2')
                return [version1, version2], f'{group_name} multi-match'

            # if simple logic
            pattern = (
                r'(?P<version>[\dxX]+(?:[.-]\s*[\dxX]+)*)'  # Version with digits/x and separators
            )
            matched = re.search(pattern, version_str, re.IGNORECASE)
            return [extract_version(matched)], f'{group_name} group'

    # after, including
    group_name = 'after'
    group_words = ['after', '>=']
    for group_wrd in group_words:
        if group_wrd in version_str:
            # More complicated multi version logic
            # '4.2.x before 4.2.8'
            multi_version_pattern = r'(?P<version1>[\dxX]+(?:\s*[.-]\s*[\dxX]+)*)\s*' \
                        r'(?:older|after|\>\=)\s*' \
                        r'(?P<version2>[\dxX]+(?:\s*[.-]\s*[\dxX]+)*)'
            multi_match = re.search(multi_version_pattern, version_str, re.IGNORECASE)
            if multi_match:
                version1 = multi_match.group('version1')
                version2 = multi_match.group('version2')
                return [version1, version2], f'{group_name} multi-match'

            # if simple logic
            pattern = (
                r'(?P<version>[\dxX]+(?:[.-]\s*[\dxX]+)*)'  # Version with digits/x and separators
            )
            matched = re.search(pattern, version_str, re.IGNORECASE)
            return [extract_version(matched)], f'{group_name} group'

    # between
    group_name = 'between'
    group_words = ['between', 'to', ' - ']
    for group_wrd in group_words:
        if group_wrd in version_str:
            # More complicated multi version logic
            # '4.2.x before 4.2.8'
            multi_version_pattern = r'(?P<version1>[\dxX]+(?:\s*[.-]\s*[\dxX]+)*)\s*' \
                        r'(?:between|to)\s*' \
                        r'(?P<version2>[\dxX]+(?:\s*[.-]\s*[\dxX]+)*)'
            multi_match = re.search(multi_version_pattern, version_str, re.IGNORECASE)
            if multi_match:
                version1 = multi_match.group('version1')
                version2 = multi_match.group('version2')
                return [version1, version2], f'{group_name} multi-match'

            # # if simple logic
            # pattern = (
            #     r'(?P<version>[\dxX]+(?:[.-]\s*[\dxX]+)\s*)'  # Version with digits/x and separators
            # )
            # matched = re.search(pattern, version_str, re.IGNORECASE)
            # return extract_version(matched), f'{group_name} group'



    pattern = (
        r'(?:v|version)?\s*'  # Optional 'v' or 'version'
        r'(?P<version>[\dxX]+(?:[.-]\s*[\dxX]+)*)'  # Version with digits/x and separators
    )
    matched = re.search(pattern, version_str, re.IGNORECASE)
    return [extract_version(matched)], 'other'

In [ ]:
test_versions = [
    'a.b.c',
    '2.0.1',
    '2.1.1.',
    'before 2. 0. 1',
    'before 2.0.1',
    'version 2.0.1',
    '1.0.4 and earlier',
    '4.2.x before 4.2.8',
    'through 1.2.0.',
    '2.1.x before 2.1.3',
    'v4.0.13.0 and earlier',
    '2.1 through 3.17',
    '2.4.30 through 2.4.55',
    '< 2.0.',
    '<=\xa02.1.4',
    '>=1.2',
    'prior to 1.3',
    'v6.9 and older ',
    'before 5.0.0-88,',
    '3.x before 3.1.1',
    'up to 29.0.x.',
    '2.0.0 to 2.1.7p1',
    'through 4.4.3.3.',
    'up to 0.4.38.'
]

In [ ]:
for x in test_versions:
    res = classify_version_string(x)
    print(f'original: {x},  extracted: {res}')
    # print(f'result versions: {generate_versions(*res)}')
    print('\n\n' + '*'*100 + '\n\n')

In [ ]:
import re
import ast
from itertools import product

def parse_version(version_str):
    components = re.findall(r'\d+|x', version_str, re.IGNORECASE)
    parsed = []
    for c in components:
        if c.lower() == 'x':
            parsed.append('x')
        else:
            parsed.append(int(c))
    # later, add 4th element to version?
    # while len(parsed) < 3:
    #     parsed.append(0)
    # if len(parsed) > 3:
    #     parsed.pop()
    return parsed

In [ ]:
classify_version_string('v5.1')

(['5.1'], 'other')

In [ ]:
pattern = (
    r'(?P<version>[\dxX]+(?:[.-]\s*[\dxX]+)*)\s*'  # Version with digits/x and separators
)
matched = re.search(pattern, 'before 1.0.2', re.IGNORECASE)
print(matched)

<re.Match object; span=(7, 12), match='1.0.2'>


In [ ]:
parse_version('v5.1')

[5, 1]

In [ ]:
def generate_versions(versions, group_name, debug=False):
    if group_name == 'other':
        result = [versions[0]]
        version_other = parse_version(versions[0])
        while len(version_other) != 3:
            if len(version_other) > 3:
                result.append('.'.join([str(x) for x in version_other]))
                version_other.pop()
            elif len(version_other) < 3:
                result.append('.'.join([str(x) for x in version_other]))
                version_other.append(0)
        else:
            result.append('.'.join([str(x) for x in version_other]))
        # print(f'result: {result}')
        # print(f'other versions: {other_versions}')
        # result_merged = result + other_versions
        # print(f'joined: {result_merged}')
        return result
        # return versions
    group_type = group_name.split()[0].lower()

    if len(versions) == 1:
        if group_type == 'before':
            return generate_versions(['0.0.0', versions[0]], 'before multi-match', debug=debug)
        elif group_type == 'through':
            return generate_versions(['0.0.0', versions[0]], 'through multi-match', debug=debug)
        # here access DB and query max version?
        elif group_type == 'after':
            return generate_versions([versions[0], '20.0.0'], 'after multi-match', debug=debug)
        else:
            return []
    elif len(versions) >= 1:
        # for ['3.x', '3.1.1']
        # 3.x
        start = parse_version(versions[0])
        len_original_start = len(start)
        # 3.1.1
        end = parse_version(versions[1])
        len_original_end = len(end)
        # normalize versions
        while len(start) != 3:
            if len(start) > 3:
                start.pop()
            elif len(start) < 3:
                start.append(0)

        while len(end) != 3:
            if len(end) > 3:
                end.pop()
            elif len(end) < 3:
                end.append(0)

        possible_values = []
        if debug:
            print(f'start version: {start}, end version: {end}')
            print(f'len_original_end: {len_original_end}')
        for i in range(3):
            # print(f'possible values: {possible_values}')
            # 3
            start_comp = start[i]
            # 3
            end_comp = end[i]
            if debug:
                print(f'Start component: {start_comp}, End component: {end_comp}')

            if start_comp == 'x':
                # Надо как-то проверять, нужно ли генерировать такик большие числа версий
                if 'before' in group_type:
                    max_val = end_comp - 1 if isinstance(end_comp, int) else 99
                else:
                    max_val = end_comp if isinstance(end_comp, int) else 99
                possible_values.append(list(range(0, max_val + 1)))

                continue
            if isinstance(start_comp, int):
                if isinstance(end_comp, str) and end_comp.lower() == 'x':
                    end_comp = 99  # High maximum for 'x' in end
                if start_comp > end_comp:
                    return []
                if start_comp < end_comp:
                    if 'before' in group_type:
                        current_max = end_comp - 1
                    else:
                        current_max = end_comp
                    # possible_values.append(list(range(start_comp, current_max + 1)))
                    possible_values.append(list(range(start_comp, 10)))

                    # Allow any values for remaining components
                    for j in range(i + 1, 3):
                        possible_values.append(list(range(0, 100)))  # Arbitrary high limit
                    break
                else:
                    possible_values.append([start_comp])
            else:
                # print(f'possible values: {possible_values}')
                possible_values.append([0])


        if debug:
            print(f'possible values: {possible_values}')
        if 'x' not in end and 'x' not in start:
            generated_components = list(product(*possible_values))
            if debug:
                print(f'generated components: {generated_components[:10]}')
            generated_components_to_use = []
            for val in generated_components:
                if not (((val[0] == end[0]
                        and val[1] > end[1]) or
                        (val[0] == end[0]
                        and val[1] == end[1]
                        and val[2] > end[2]) or
                        val[0] > end[0])
                    or ((val[0] == start[0]
                         and val[1] < start[1]) or
                        (val[0] == start[0]
                         and val[1] == start[1]
                         and val[2] < start[2])) or
                        val[0] < start[0]):
                    generated_components_to_use.append(val)
            if debug:
                print(f'generated components to use: {generated_components_to_use[:10], generated_components_to_use[-10:]}')
            versions_list = ['.'.join(map(str, v)) for v in generated_components_to_use]

            # return versions_list
        else:
            generated_components = list(product(*possible_values))
            versions_list = ['.'.join(map(str, v)) for v in generated_components]
        if len_original_end == 2 or len_original_start == 2:
            versions_set = []
            for x in versions_list:
                versions_set.append(x.split('.')[:2])
            versions_set = set(['.'.join(y) for y in versions_set])
            # for x in versions_list:
            versions_list.extend(list(versions_set))
        if debug:
            print(versions_list[:10])
        return versions_list
    else:
        # print('last else')
        return []

In [ ]:
ver = classify_version_string('1.2')
rrr = generate_versions(ver[0], ver[1], debug=1)
print(ver)
print(rrr)

(['1.2'], 'other')
['1.2', '1.2', '1.2.0']


In [ ]:
ver = classify_version_string('v6')
rrr = generate_versions(ver[0], ver[1], debug=1)
print(ver)

print(rrr)

(['6'], 'other')
['6', '6', '6.0', '6.0.0']


In [ ]:
from collections import Counter
rr = generate_versions(['1.12'], 'before group', debug=0)
Counter(len(x) for x in rr)
# list(filter(lambda x: len(x) == 4, rr))

Counter({5: 200, 6: 2721, 7: 8280, 4: 93, 3: 20})

In [ ]:
set([1, 2])

{1, 2}

In [ ]:
r = []
d_vers = {}
d = {}
for i, row in df_test.iterrows():
    possible_versions = []
    # print(i, row['version_ner'])
    for version_ner in row['version_ner']:
        # print(version_ner)
        preprocessed_ner = classify_version_string(version_ner)
        if preprocessed_ner[0][0] is None:
            continue
        # print(preprocessed_ner, end='\n'+'*'*50+'\n')
        generated_versions = generate_versions(*preprocessed_ner)
        possible_versions.extend(generated_versions)
    r.append(1 if row['version'] in possible_versions else 0)
    d_vers[(row['version'], row['cve_id'])] = row['version'] if row['version'] in possible_versions else 0
    d[row['cve_id']] = possible_versions

In [ ]:
sum(r)

55

In [ ]:
df_test[df_test.cve_id == 'CVE-2022-47595']

,cve_id_pk,cve_id,descr_id,cve_id_fk,descr,cve_id_fk.1,cpe_id_fk,config_id,cpe_id_pk,cpe_version,part,vendor,product,version,update,edition,sw_edition,target_sw,target_hw,language,other,initial_cpe,cve_year,ners_list,scores_list,vendor_ner,product_ner,version_ner,vendor_score_ner,product_score_ner,version_score_ner,dedup_vendor,dedup_vendor_score,dedup_product,dedup_product_score
4,34073,CVE-2022-47595,175843,34073,Improper Limitation of a Pathname to a Restricted Directory ('Path Traversal') vulnerability in ...,34073,216534,486886,216534,2.3,a,codecabin,wp_go_maps,9.0.12,NaN,NaN,NaN,pro,wordpress,NaN,NaN,cpe:2.3:a:codecabin:wp_go_maps:9.0.12:*:*:*:pro:wordpress:*:*,2022,"[[], [wp go maps, wp google maps)], []]","[[], [0.9817508, 0.9798541], []]",[],"[wp go maps, wp google maps)]",[],[],"[0.9817508, 0.9798541]",[],[],[],[wp go maps],[0.9817508]


In [ ]:
df_test[df_test.cve_id == 'CVE-2022-47595'][['cve_id', 'descr', 'initial_cpe', 'version', 'version_ner']]

,cve_id,descr,initial_cpe,version,version_ner
4,CVE-2022-47595,Improper Limitation of a Pathname to a Restricted Directory ('Path Traversal') vulnerability in ...,cpe:2.3:a:codecabin:wp_go_maps:9.0.12:*:*:*:pro:wordpress:*:*,9.0.12,[]


In [ ]:
pprint({x:y for x,y in d_vers.items() if y == 0})

{('-', 'CVE-2015-3374'): 0,
 ('-', 'CVE-2017-1000212'): 0,
 ('-', 'CVE-2020-12695'): 0,
 ('-', 'CVE-2020-3622'): 0,
 ('-', 'CVE-2021-3438'): 0,
 ('-', 'CVE-2023-3160'): 0,
 ('0.0.3', 'CVE-2022-25900'): 0,
 ('0.0.30', 'CVE-2020-27589'): 0,
 ('0.17.2', 'CVE-2022-39255'): 0,
 ('0.34.8', 'CVE-2023-50709'): 0,
 ('0.6.0', 'CVE-2021-33767'): 0,
 ('0.9.5', 'CVE-2023-27522'): 0,
 ('08_00_01', 'CVE-2007-6145'): 0,
 ('1.1.70', 'CVE-2023-41891'): 0,
 ('1.10.0', 'CVE-2022-0944'): 0,
 ('1.242.0', 'CVE-2023-34927'): 0,
 ('1.3.6', 'CVE-2023-1404'): 0,
 ('12.04', 'CVE-2022-23935'): 0,
 ('17.3.0', 'CVE-2022-4731'): 0,
 (nan, 'CVE-2022-22508'): 0,
 ('1.1.113', 'CVE-2022-2223'): 0,
 ('1.28.8', 'CVE-2022-2582'): 0,
 ('1.3.2', 'CVE-2022-35918'): 0,
 ('11.2.7', 'CVE-2020-10683'): 0,
 ('12.2\\(14\\)za4', 'CVE-2016-1409'): 0,
 ('12.2\\(15\\)cz', 'CVE-2016-1409'): 0,
 ('12.3\\(7\\)xi8c', 'CVE-2016-1409'): 0,
 ('2.1.0', 'CVE-2023-33972'): 0,
 ('2.4.2.6', 'CVE-2022-21167'): 0,
 ('2019.06', 'CVE-2021-30141'): 0,
 

In [ ]:
df_test.sample(2)

,cve_id_pk,cve_id,descr_id,cve_id_fk,descr,cve_id_fk.1,cpe_id_fk,config_id,cpe_id_pk,cpe_version,part,vendor,product,version,update,edition,sw_edition,target_sw,target_hw,language,other,initial_cpe,cve_year,ners_list,scores_list,vendor_ner,product_ner,version_ner,vendor_score_ner,product_score_ner,version_score_ner,dedup_vendor,dedup_vendor_score,dedup_product,dedup_product_score
30,84864,CVE-2020-21087,144089,84864,Cross Site Scripting (XSS) in X2Engine X2CRM v6.9 and older allows remote attackers to execute a...,84864,468414,174214,468414,2.3,a,x2engine,x2crm,5.2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,cpe:2.3:a:x2engine:x2crm:5.2:*:*:*:*:*:*:*,2020,"[[x2engine], [x2crm], [v6.9 and older]]","[[0.948784], [0.9462559], [0.9998064]]",[x2engine],[x2crm],[v6.9 and older],[0.948784],[0.9462559],[0.9998064],[x2engine],[0.948784],[x2crm],[0.9462559]
35,25172,CVE-2022-3069,166942,25172,"The WordLift WordPress plugin before 3.37.2 does not sanitise and escape its settings, allowing ...",25172,152412,265810,152412,2.3,a,wordlift,wordlift,3.33.0,NaN,NaN,NaN,NaN,wordpress,NaN,NaN,cpe:2.3:a:wordlift:wordlift:3.33.0:*:*:*:*:wordpress:*:*,2022,"[[], [wordlift, wordpress], [before 3.37.2]]","[[], [0.99641657, 0.9202705], [0.9999022]]",[],"[wordlift, wordpress]",[before 3.37.2],[],"[0.99641657, 0.9202705]",[0.9999022],[],[],[wordlift],[0.99641657]


In [ ]:
# собираем vendor:product:version
created_cpe = {}
for i, row in df_test.iterrows():
    possible_versions = []
    # print(i, row['version_ner'])
    for version_ner in row['version_ner']:
        # print(version_ner)
        preprocessed_ner = classify_version_string(version_ner)
        if preprocessed_ner[0][0] is None:
            continue
        # print(preprocessed_ner, end='\n'+'*'*50+'\n')
        generated_versions = generate_versions(*preprocessed_ner)
        possible_versions.extend(generated_versions)
    # r.append(1 if row['version'] in possible_versions else 0)
    # d_vers[(row['version'], row['cve_id'])] = row['version'] if row['version'] in possible_versions else 0
    cpe_with_versions = []
    cpe = ''
    if row['dedup_vendor']:
        cpe += row['dedup_vendor'][0].replace(' ', '_')
    cpe += ':'
    if row['dedup_product']:
        cpe += row['dedup_product'][0].replace(' ', '_')
    cpe += ':'
    for version in possible_versions:
        cpe_with_version = cpe + version
        cpe_with_versions.append(cpe_with_version)
    created_cpe[row['cve_id']] = cpe_with_versions

In [ ]:
created_cpe.keys()

dict_keys(['CVE-2023-49619', 'CVE-2023-40347', 'CVE-2002-2220', 'CVE-2016-0851', 'CVE-2022-47595', 'CVE-2020-3622', 'CVE-2022-28755', 'CVE-2023-51508', 'CVE-2021-3806', 'CVE-2023-33972', 'CVE-2024-23659', 'CVE-2018-19156', 'CVE-2022-29524', 'CVE-2006-3041', 'CVE-2023-46624', 'CVE-2016-1409', 'CVE-2023-27522', 'CVE-2009-2649', 'CVE-2020-10683', 'CVE-2023-35094', 'CVE-2020-10941', 'CVE-2017-17707', 'CVE-2006-0248', 'CVE-2018-7445', 'CVE-2022-39255', 'CVE-2017-1000083', 'CVE-2004-2437', 'CVE-2021-3438', 'CVE-2021-40898', 'CVE-2021-30141', 'CVE-2020-21087', 'CVE-2021-25982', 'CVE-2018-19115', 'CVE-2023-50709', 'CVE-2008-2547', 'CVE-2022-3069', 'CVE-2021-33767', 'CVE-2023-41891', 'CVE-2020-9114', 'CVE-2020-25494', 'CVE-2022-25900', 'CVE-2023-32743', 'CVE-2016-5843', 'CVE-2022-22508', 'CVE-2005-0005', 'CVE-2022-3205', 'CVE-2023-30621', 'CVE-2022-34191', 'CVE-2019-12491', 'CVE-2022-31507', 'CVE-2021-30134', 'CVE-2015-9231', 'CVE-2022-23935', 'CVE-2016-5007', 'CVE-2022-22116', 'CVE-2022-4731',

In [ ]:
gg = df_test[(df_test.dedup_vendor.astype(str) != '[]') & (df_test.dedup_product.astype(str) != '[]')].head()
gg

,cve_id_pk,cve_id,descr_id,cve_id_fk,descr,cve_id_fk.1,cpe_id_fk,config_id,cpe_id_pk,cpe_version,part,vendor,product,version,update,edition,sw_edition,target_sw,target_hw,language,other,initial_cpe,cve_year,ners_list,scores_list,vendor_ner,product_ner,version_ner,vendor_score_ner,product_score_ner,version_score_ner,dedup_vendor,dedup_vendor_score,dedup_product,dedup_product_score
9,60587,CVE-2023-33972,11973,60587,"Scylladb is a NoSQL data store using the seastar framework, compatible with Apache Cassandra. Au...",60587,349826,547943,349826,2.3,a,scylladb,scylladb,2.1.0,rc2,NaN,NaN,NaN,NaN,NaN,NaN,cpe:2.3:a:scylladb:scylladb:2.1.0:rc2:*:*:*:*:*:*,2023,"[[apache], [scylladb], []]","[[0.5997038], [0.8270666], []]",[apache],[scylladb],[],[0.5997038],[0.8270666],[],[apache],[0.5997038],[scylladb],[0.8270666]
13,122032,CVE-2006-3041,154831,122032,PHP remote file inclusion vulnerability in Ltwcalendar/calendar.php in Codewalkers Ltwcalendar 4...,122032,591250,30619,591250,2.3,a,codewalkers,ltwcalendar,4.1.3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,cpe:2.3:a:codewalkers:ltwcalendar:4.1.3:*:*:*:*:*:*:*,2006,"[[codewalkers], [ltwcalendar], [4.1.3]]","[[0.80653185], [0.99936587], [0.99988186]]",[codewalkers],[ltwcalendar],[4.1.3],[0.80653185],[0.99936587],[0.99988186],[codewalkers],[0.80653185],[ltwcalendar],[0.99936587]
16,57058,CVE-2023-27522,8444,57058,HTTP Response Smuggling vulnerability in Apache HTTP Server via mod_proxy_uwsgi. This issue affe...,57058,326848,324077,326848,2.3,a,unbit,uwsgi,0.9.5,-,NaN,NaN,NaN,NaN,NaN,NaN,cpe:2.3:a:unbit:uwsgi:0.9.5:-:*:*:*:*:*:*,2023,"[[apache, apache], [http server], [2.4.30 through 2.4.55.\n\nspecial]]","[[0.999485, 0.99801874], [0.6676416], [0.9999151]]","[apache, apache]",[http server],[2.4.30 through 2.4.55.\n\nspecial],"[0.999485, 0.99801874]",[0.6676416],[0.9999151],[apache],[0.999485],[http server],[0.6676416]
22,119362,CVE-2006-0248,152161,119362,"Virata-EmWeb web server 6_1_0, as used in (1) Intracom JetSpeed 500 and 520 and (2) Allied Data ...",119362,588119,3608,588119,2.3,a,intracom,jetspeed,500,NaN,NaN,NaN,NaN,NaN,NaN,NaN,cpe:2.3:a:intracom:jetspeed:500:*:*:*:*:*:*:*,2006,"[[intracom], [jetspeed], [500, 520, 811]]","[[0.9978908], [0.6977878], [0.8618722, 0.83917147, 0.91485614]]",[intracom],[jetspeed],"[500, 520, 811]",[0.9978908],[0.6977878],"[0.8618722, 0.83917147, 0.91485614]",[intracom],[0.9978908],[jetspeed],[0.6977878]
25,142268,CVE-2017-1000083,128110,142268,backend/comics/comics-document.c (aka the comic book backend) in GNOME Evince before 3.24.1 allo...,142268,655153,32189,655153,2.3,a,gnome,evince,3.7.5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,cpe:2.3:a:gnome:evince:3.7.5:*:*:*:*:*:*:*,2017,"[[gnome], [evince], [before 3.24.1]]","[[0.99943835], [0.9993463], [0.99992555]]",[gnome],[evince],[before 3.24.1],[0.99943835],[0.9993463],[0.99992555],[gnome],[0.99943835],[evince],[0.9993463]


In [ ]:
for k,v in created_cpe.items():
    if k in gg.cve_id.values:
        print(k, v[:5])

CVE-2023-33972 []
CVE-2006-3041 ['codewalkers:ltwcalendar:4.1.3', 'codewalkers:ltwcalendar:4.1.3']
CVE-2023-27522 []
CVE-2006-0248 ['intracom:jetspeed:500', 'intracom:jetspeed:500', 'intracom:jetspeed:500.0', 'intracom:jetspeed:500.0.0', 'intracom:jetspeed:520']
CVE-2017-1000083 ['gnome:evince:0.0.0', 'gnome:evince:0.0.1', 'gnome:evince:0.0.2', 'gnome:evince:0.0.3', 'gnome:evince:0.0.4']


In [ ]:
created_cpe['CVE-2020-21087']

['x2engine:x2crm:0.0.0',
 'x2engine:x2crm:0.0.1',
 'x2engine:x2crm:0.0.2',
 'x2engine:x2crm:0.0.3',
 'x2engine:x2crm:0.0.4',
 'x2engine:x2crm:0.0.5',
 'x2engine:x2crm:0.0.6',
 'x2engine:x2crm:0.0.7',
 'x2engine:x2crm:0.0.8',
 'x2engine:x2crm:0.0.9',
 'x2engine:x2crm:0.0.10',
 'x2engine:x2crm:0.0.11',
 'x2engine:x2crm:0.0.12',
 'x2engine:x2crm:0.0.13',
 'x2engine:x2crm:0.0.14',
 'x2engine:x2crm:0.0.15',
 'x2engine:x2crm:0.0.16',
 'x2engine:x2crm:0.0.17',
 'x2engine:x2crm:0.0.18',
 'x2engine:x2crm:0.0.19',
 'x2engine:x2crm:0.0.20',
 'x2engine:x2crm:0.0.21',
 'x2engine:x2crm:0.0.22',
 'x2engine:x2crm:0.0.23',
 'x2engine:x2crm:0.0.24',
 'x2engine:x2crm:0.0.25',
 'x2engine:x2crm:0.0.26',
 'x2engine:x2crm:0.0.27',
 'x2engine:x2crm:0.0.28',
 'x2engine:x2crm:0.0.29',
 'x2engine:x2crm:0.0.30',
 'x2engine:x2crm:0.0.31',
 'x2engine:x2crm:0.0.32',
 'x2engine:x2crm:0.0.33',
 'x2engine:x2crm:0.0.34',
 'x2engine:x2crm:0.0.35',
 'x2engine:x2crm:0.0.36',
 'x2engine:x2crm:0.0.37',
 'x2engine:x2crm:0.0.3

## Пробуем еще не обработанные версии из NVD

In [ ]:
cves = ['CVE-2025-43704', 'CVE-2025-2073']
descrs = ['Arctera/Veritas Data Insight before 7.1.2 can send cleartext credentials when configured to use HTTP Basic Authentication to a Dell Isilon OneFS server.',
          'Out-of-Bounds Read in ip_set_bitmap_ip.c in Google ChromeOS Kernel Versions 6.1, 5.15, 5.10, 5.4, 4.19. on All devices where Termina is used allows an attacker with CAP_NET_ADMIN privileges to cause memory corruption and potentially escalate privileges via crafted ipset commands.']

In [ ]:
df_new_cves = pd.DataFrame(data={'cve_id': cves,
                                 'descr': descrs})

In [ ]:
!unzip nvdcve-1.1-2025.json.zip

Archive:  nvdcve-1.1-2025.json.zip
  inflating: nvdcve-1.1-2025.json    


In [ ]:
import json
with open('/content/nvdcve-1.1-2025.json') as f:
    data = json.load(f)

In [ ]:
rows = []
for i, item in enumerate(data['CVE_Items']):
    if i < 300:
        cve_id = item['cve']['CVE_data_meta']['ID']
        desc_list = item['cve']['description']['description_data']
        # Extract the first English description
        description = next((desc['value'] for desc in desc_list if desc['lang'] == 'en'), None)
        rows.append({'cve_id': cve_id, 'descr': description})

df_new_cves = pd.DataFrame(rows, columns=['cve_id', 'descr'])
df_new_cves = df_new_cves.sample(30)

In [ ]:
df_new_cves[['ners_list', 'scores_list']] = df_new_cves['descr'].apply(lambda x: extract_ners(x)).apply(pd.Series)
df_new_cves['vendor_ner'], df_new_cves['product_ner'], df_new_cves['version_ner'] = zip(*df_new_cves['ners_list'])
df_new_cves['vendor_score_ner'], df_new_cves['product_score_ner'], df_new_cves['version_score_ner'] = zip(*df_new_cves['scores_list'])

Device set to use cpu
/usr/local/lib/python3.11/dist-packages/transformers/pipelines/token_classification.py:398: UserWarning: Tokenizer does not support real words, using fallback heuristic
  warnings.warn(
Device set to use cpu
Device set to use cpu
Device set to use cpu
Device set to use cpu
Device set to use cpu
Device set to use cpu
Device set to use cpu
Device set to use cpu
Device set to use cpu
Device set to use cpu
Device set to use cpu
Device set to use cpu
Device set to use cpu
Device set to use cpu
Device set to use cpu
Device set to use cpu
Device set to use cpu
Device set to use cpu
Device set to use cpu
Device set to use cpu
Device set to use cpu
Device set to use cpu
Device set to use cpu
Device set to use cpu
Device set to use cpu
Device set to use cpu
Device set to use cpu
Device set to use cpu
Device set to use cpu


In [ ]:
df_new_cves[(df_new_cves.vendor_ner.astype(str) == '[]') & (df_new_cves.product_ner.astype(str) == '[]')]

,cve_id,descr,ners_list,scores_list,vendor_ner,product_ner,version_ner,vendor_score_ner,product_score_ner,version_score_ner
81,CVE-2025-0182,A vulnerability in danswer-ai/danswer version 0.9.0 allows for denial of service through memory ...,"[[], [], [0.9.0, 0.115.3.]]","[[], [], [0.9998215, 0.9996768]]",[],[],"[0.9.0, 0.115.3.]",[],[],"[0.9998215, 0.9996768]"
258,CVE-2025-0424,"In the ""bestinformed Web"" application, some user input was not properly sanitized. This leads to...","[[], [], []]","[[], [], []]",[],[],[],[],[],[]
266,CVE-2025-0433,"The Master Addons – Elementor Addons with White Label, Free Widgets, Hover Effects, Conditions, ...","[[], [], [2.0.7.1]]","[[], [], [0.99983406]]",[],[],[2.0.7.1],[],[],[0.99983406]
87,CVE-2025-0189,"In version 3.25.0 of aimhubio/aim, the tracking server is vulnerable to a denial of service atta...","[[], [], [3.25.0]]","[[], [], [0.999676]]",[],[],[3.25.0],[],[],[0.999676]
179,CVE-2025-0315,A vulnerability in ollama/ollama <=0.3.14 allows a malicious user to create a customized GGUF mo...,"[[], [], [<=0.3.14]]","[[], [], [0.9982821]]",[],[],[<=0.3.14],[],[],[0.9982821]


In [ ]:
s = 'parse/Csv2_parse.c in MaraDNS 1.3.03, and other versions before 1.4.03, does not properly handle hostnames that do not end in a "." (dot) character, which allows remote attackers to cause a denial of service (NULL pointer dereference) via a crafted csv2 zone file.'

In [ ]:
token_classifier = pipeline(
    "token-classification", model=final_model, aggregation_strategy="first", tokenizer=final_tokenizer
)
res = token_classifier(s)
for i, r in enumerate(res):
    # print('Entity: '+ r['entity_group'] + '   Word: ' + r['word'])
    print('Entity: '+ r['entity_group'] + '   Word: ' + r['word'] + '. Probs:  ' + str(round(r['score'], 4)))


Device set to use cpu


Entity: product   Word:  MaraDNS. Probs:  0.9997
Entity: version   Word:  1.3.03,. Probs:  0.9999
Entity: version   Word:  before 1.4.03,. Probs:  0.9999


In [ ]:
import torch
from transformers import AutoModelForTokenClassification, AutoTokenizer

# Assuming final_model and final_tokenizer are already loaded
# final_model = AutoModelForTokenClassification.from_pretrained(...)
# final_tokenizer = AutoTokenizer.from_pretrained(...)

def get_token_probabilities(text, model, tokenizer):
    # Tokenize input
    inputs = tokenizer(text, return_tensors="pt", padding=True, truncation=True).to('cuda')

    # Get model predictions
    with torch.no_grad():
        outputs = model(**inputs)

    # Get logits and apply softmax to get probabilities
    logits = outputs.logits
    probabilities = torch.nn.functional.softmax(logits, dim=-1)

    # Convert input IDs to tokens
    tokens = tokenizer.convert_ids_to_tokens(inputs["input_ids"][0])

    # Get label names from model config
    id2label = model.config.id2label

    # Prepare results
    results = []
    for token_idx, (token, prob) in enumerate(zip(tokens, probabilities[0])):
        token_probs = {
            id2label[label_id]: f"{prob[label_id].item():.4f}"
            for label_id in range(len(id2label))
        }
        results.append({
            "token": token,
            "token_id": token_idx,
            "probabilities": token_probs
        })

    return results

# Usage
token_probabilities = get_token_probabilities(s, nuner_model, nuner_tokenizer)

# Print results
for item in token_probabilities:
    print(f"Token: {item['token']} ({item['token_id']})")
    for label, prob in item['probabilities'].items():
        print(f"  {label}: {prob}")
    print()

Token: <s> (0)
  O: 0.9972
  B-product: 0.0000
  I-product: 0.0000
  B-vendor: 0.0000
  I-vendor: 0.0000
  B-version: 0.0027
  I-version: 0.0000

Token: Ġ (1)
  O: 0.9965
  B-product: 0.0004
  I-product: 0.0000
  B-vendor: 0.0001
  I-vendor: 0.0000
  B-version: 0.0029
  I-version: 0.0001

Token: Ċ (2)
  O: 0.9995
  B-product: 0.0001
  I-product: 0.0000
  B-vendor: 0.0000
  I-vendor: 0.0000
  B-version: 0.0003
  I-version: 0.0000

Token: A (3)
  O: 0.9999
  B-product: 0.0000
  I-product: 0.0000
  B-vendor: 0.0000
  I-vendor: 0.0000
  B-version: 0.0000
  I-version: 0.0000

Token: Ġvulnerability (4)
  O: 1.0000
  B-product: 0.0000
  I-product: 0.0000
  B-vendor: 0.0000
  I-vendor: 0.0000
  B-version: 0.0000
  I-version: 0.0000

Token: , (5)
  O: 0.9999
  B-product: 0.0000
  I-product: 0.0000
  B-vendor: 0.0000
  I-vendor: 0.0000
  B-version: 0.0001
  I-version: 0.0000

Token: Ġwhich (6)
  O: 0.9999
  B-product: 0.0000
  I-product: 0.0000
  B-vendor: 0.0000
  I-vendor: 0.0000
  B-version: 

# Train final model on full dataset

Объединяем все датасеты в один

In [ ]:
# этот способ перестал работать: видимо, что-то с версиями
# ds_train = load_dataset("/content/drive/MyDrive/vkr_pandan/dataset/230425")
ds_test = load_from_disk('/content/drive/MyDrive/vkr_pandan/dataset/230425_test')

In [ ]:
# создаем тренировочный датасет
ds_tr = load_from_disk('/content/drive/MyDrive/vkr_pandan/dataset/230425/train')
ds_te = load_from_disk('/content/drive/MyDrive/vkr_pandan/dataset/230425/test')
ds_train = datasets.DatasetDict({"train":ds_tr,"test":ds_te})

In [ ]:
ds_all = concatenate_datasets([ds_train['train'], ds_train['test'], ds_test])

/usr/local/lib/python3.11/dist-packages/datasets/table.py:1421: FutureWarning: promote has been superseded by promote_options='default'.
  table = cls._concat_blocks(blocks, axis=0)


In [ ]:
lbls_in_dataset = [
    'O',
    'B-product',
    'I-product',
    'B-vendor',
    'I-vendor',
    'B-version',
    'I-version',
]
label2id = {v:i for i, v in enumerate(lbls_in_dataset)}
id2label = {i:v for i, v in enumerate(lbls_in_dataset)}


In [ ]:
model_path = "/content/drive/MyDrive/vkr_pandan/nuner_110525/best_model"
tokenizer = AutoTokenizer.from_pretrained(model_path, use_fast=True, add_prefix_space=True)
model = AutoModelForTokenClassification.from_pretrained(model_path, num_labels=len(lbls_in_dataset), id2label=id2label, label2id=label2id)

In [ ]:
def tokenize_and_align_labels(examples, tokenizer):
    tokenized_inputs = tokenizer(examples['tokens'], truncation=True, is_split_into_words=True)

    labels = []
    for i, label in enumerate(examples['ner_tags']):
        word_ids = tokenized_inputs.word_ids(batch_index=i)
        previous_index = None
        label_ids = []
        for word_idx in word_ids:
            if word_idx is None:
                label_ids.append(-100)
            else:
                label_ids.append(label[word_idx])
        labels.append(label_ids)

    tokenized_inputs['labels'] = labels
    return tokenized_inputs



In [ ]:
tokenized_ds_all = ds_all.map(lambda x: tokenize_and_align_labels(x, tokenizer), batched=True)

Map:   0%|          | 0/15322 [00:00<?, ? examples/s]

In [ ]:
data_collator = DataCollatorForTokenClassification(tokenizer)

{'learning_rate': 0.00019863319858102557, 'weight_decay': 0.0861746314370445, 'num_train_epochs': 7}

In [ ]:
training_args = TrainingArguments(
        # f"{model_name}-finetuned-ner",
        output_dir='./training_output',
        overwrite_output_dir=True,
        # eval_strategy = "epoch",
        torch_compile=True,
        # 64\
        per_device_train_batch_size=32,
        # per_device_eval_batch_size=32,
        learning_rate=0.00019863319858102557,
        weight_decay=0.0861746314370445,
        num_train_epochs=7,
        seed=42,
        data_seed=24,
        gradient_accumulation_steps=2,
        warmup_ratio=0.1,
        report_to=None,
        logging_dir="./logs",
        logging_steps=1,
        # load_best_model_at_end=True,
        # metric_for_best_model="eval_f1",
        # greater_is_better=False,
        save_total_limit=1,
        save_strategy="epoch",  # Changed to match evaluation_strategy
    )

    # early_stopping = EarlyStoppingCallback(
    #     early_stopping_patience=1,  # Stop if F1 decreases for 1 consecutive epoch
    #     early_stopping_threshold=0.001
    # )

trainer = Trainer(
    model,
    training_args,
    train_dataset=tokenized_ds_all,
    # train_dataset=small_dataset_train,
    # eval_dataset=small_dataset_test,
    data_collator=data_collator,
    # compute_metrics=compute_metrics_custom,
    tokenizer=tokenizer,
    # callbacks=[early_stopping],
)

trainer.train()
save_dir = '/content/drive/MyDrive/vkr_pandan/nuner_180525_full_dataset'
trainer.save_model(save_dir)
print(f"Model saved to {save_dir}")

    # # Evaluate and save best model globally
    # eval_metrics = trainer.evaluate()
    # current_f1 = eval_metrics["eval_f1"]

    # if current_f1 > best_f1:
    #     best_f1 = current_f1
    #     trainer.save_model("./best_model")
    #     trainer.save_model(GDRIVE_DIR)
    #     printer(f"New best model saved with F1: {best_f1:.4f}")
    # return current_f1

The speedups for torchdynamo mostly come with GPU Ampere or higher and which is not detected here.
Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).
<ipython-input-16-af235f269208>:32: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(
W0518 11:32:35.309000 378 torch/_inductor/utils.py:1137] [0/0] Not enough SMs to use max_autotune_gemm mode


Step,Training Loss
1,0.012500
2,0.014600
3,0.014800
4,0.009700
5,0.009000
6,0.010500
7,0.002500
8,0.006900
9,0.008700
10,0.008100


Model saved to /content/drive/MyDrive/vkr_pandan/nuner_180525_full_dataset
